# Automated ML

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

We start by setting up our experiment in our workspace.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'cc_automl'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-135758
Azure region: southcentralus
Subscription id: 510b94ba-e453-4417-988b-fbdc37b55ca7
Resource group: aml-quickstarts-135758


## Dataset

We're using the [UCI's Machine Learning repository's Cervical cancer (Risk Factors) Data Set](https://archive.ics.uci.edu/ml/datasets/Cervical+cancer+%28Risk+Factors%29). The dataset was collected at 'Hospital Universitario de Caracas' in Caracas, Venezuela. The dataset comprises demographic information, habits, and historic medical records of 858 patients. Several patients decided not to answer some of the questions because of privacy concerns (missing values).

**Citation: Kelwin Fernandes, Jaime S. Cardoso, and Jessica Fernandes. 'Transfer Learning with Partial Observability Applied to Cervical Cancer Screening.' Iberian Conference on Pattern Recognition and Image Analysis. Springer International Publishing, 2017.**

Our target variable here is **Biopsy**. Biopsy is a sample of tissue taken from the body in order to examine it more closely. A doctor should recommend a biopsy when an initial test suggests an area of tissue in the body isn't normal. Doctors may call an area of abnormal tissue a lesion, a tumor, or a mass.
Here this categorical variable contains the *Biopsy Test Result*.

More information about how the data is cleaned and used maybe found in the `data-cleaning.ipynb` and `train.py` files.

In [3]:
ds = TabularDatasetFactory.from_delimited_files(path="https://archive.ics.uci.edu/ml/machine-learning-databases/00383/risk_factors_cervical_cancer.csv")

Here, we clean and observe our data.

In [4]:
x, y = clean_data(ds)

df = pd.concat([x, pd.DataFrame(y)], axis = 1)
df.head()

,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs:molluscum contagiosum,STDs:HIV,STDs:Hepatitis B,STDs:HPV,STDs: Number of diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Biopsy
0,18.0,4.0,15.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,15.0,1.0,14.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34.0,1.0,17.089779,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,52.0,5.0,16.000000,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,46.0,3.0,21.000000,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now we split our data in order to fe

In [5]:
outname2='training_dataset.csv'
outdir2='training/'
if not os.path.exists(outdir2):
    os.mkdir(outdir2)

df_train, df_test = train_test_split(df, test_size=0.25)

fullpath2=os.path.join(outdir2,outname2)
df_test.to_csv(fullpath2)

outname='validation_dataset.csv'
outdir='validation/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

fullpath=os.path.join(outdir,outname)
df_test.to_csv(fullpath)

Now we store our dataset in our default datastore in order to access it.

In [6]:
datastore = ws.get_default_datastore()

In [7]:
datastore.upload(src_dir = "training/", target_path = "data/")
datastore.upload(src_dir = "validation/", target_path = "data/")

Uploading an estimated of 1 files
Target already exists. Skipping upload for data/training_dataset.csv
Uploaded 0 files
Uploading an estimated of 1 files
Target already exists. Skipping upload for data/validation_dataset.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_6751b472735b449283d380ca309447b0

In [8]:
training_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/training_dataset.csv"))])
validation_data = TabularDatasetFactory.from_delimited_files(path = [(datastore, ("data/validation_dataset.csv"))])

## AutoML Configuration

We start by setting up our compute cluster, where we will run our automl run.

In [9]:
cpu_cluster_name = "cpucluster-aml"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


We've used the following configuration for our run:

|Setting |Reasons|
|-|-|
|**experiment_timeout_minutes**| Maximum amount of time in minutes that all iterations combined can take before the experiment terminates. I've taken this to be 30 mins due to the presence of 730 rows. |
|**max_concurrent_iterations**|These are the iterations occuring simultaneously and has to be equal to the number of nodes in the cluster(5-1))|
|**n_cross_validations**|Using 5 cross validations to avoi8d overfitting) |
|**primary_metric**|Since the data isn't quite balanced, Weighted Average Precision Score |
|**task**|Classification |
|**compute_target**|This is the compute cluster we will be using |
|**training_data**|This is the training dataset stored in the default datastore  |
|**label_column_name**|This is the target variable|


In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes" :30,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric": 'average_precision_score_weighted',
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    n_cross_validations=5,
    task="classification",
    primary_metric="average_precision_score_weighted",
    compute_target=cpu_cluster,
    training_data=training_data,
    label_column_name="Biopsy",
    max_cores_per_iteration=-1,
    enable_onnx_compatible_models=True
    )

Submitting the run

In [11]:
remote_run = experiment.submit(config = automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on cpucluster-aml with default configuration
Running on remote compute: cpucluster-aml
Parent Run ID: AutoML_036c5960-9124-4360-b32c-e1c6a0dca31a

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+-------------------

## Run Details
The `Rundetails` widget, as the name suggests gives us greater insight about how the Run is proceeding, enabling us to monitor and understand the situation, thereby dealing with it accordingly.

In [12]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

The best performing model is the `VotingEnsemble` with a score of 0.9006. It maybe observed to be derived from the following:

|**Field**|Value|
|-|-|
|**Ensembled Iterations**|0, 14, 15, 6, 26|
|**Ensembled Algorithms**|'LightGBM', 'RandomForest', 'XGBoostClassifier', 'ExtremeRandomTrees', 'XGBoostClassifier'|
|**Ensemble Weights**|0.3333333333333333, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666|
|**Best Individual Pipeline Score**|"0.9005922928114609"|

In [15]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

model_ml = best_run.register_model(model_name='Cervical_Cancer_prediction_Biopsy_auto_ml', model_path='./')

Package:azureml-automl-runtime, training version:1.20.0, current version:1.19.0
Package:azureml-core, training version:1.20.0, current version:1.19.0
Package:azureml-dataprep, training version:2.7.2, current version:2.6.1
Package:azureml-dataprep-native, training version:27.0.0, current version:26.0.0
Package:azureml-dataprep-rslex, training version:1.5.0, current version:1.4.0
Package:azureml-dataset-runtime, training version:1.20.0, current version:1.19.0.post1
Package:azureml-defaults, training version:1.20.0, current version:1.19.0
Package:azureml-interpret, training version:1.20.0, current version:1.19.0
Package:azureml-pipeline-core, training version:1.20.0, current version:1.19.0
Package:azureml-telemetry, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-client, training version:1.20.0, current version:1.19.0
Package:azureml-train-automl-runtime, training version:1.20.0, current version:1.19.0


Run(Experiment: cc_automl,
Id: AutoML_036c5960-9124-4360-b32c-e1c6a0dca31a_28,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                               objective='reg:logistic',
                                                                                               random_state=0,
                                     

## Retrieve and Save ONNX Model


In [41]:
from azureml.automl.runtime.onnx_convert import OnnxConverter

b_run , onnx_mdl = remote_run.get_output(return_onnx_model=True)
onnx_fl_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(onnx_mdl, onnx_fl_path)

### Predict with the ONNX model, using onnxruntime package

In [48]:
import sys
import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants

if sys.version_info < OnnxConvertConstants.OnnxIncompatiblePythonVersion:
    python_version_compatible = True
else:
    python_version_compatible = False

import onnxruntime
from azureml.automl.runtime.onnx_convert import OnnxInferenceHelper

def get_onnx_res(run):
    res_path = 'onnx_resource.json'
    run.download_file(name=constants.MODEL_RESOURCE_PATH_ONNX, output_file_path=res_path)
    with open(res_path) as f:
        onnx_res = json.load(f)
    return onnx_res

if python_version_compatible:
    mdl_bytes = onnx_mdl.SerializeToString()
    onnx_res = get_onnx_res(b_run)

    df_test['Column1'] = 0.0
    onnxrt_helper = OnnxInferenceHelper(mdl_bytes, onnx_res)
    pred_onnx, pred_prob_onnx = onnxrt_helper.predict(df_test)

    print(pred_onnx)
    print(pred_prob_onnx)
else:
    print('Please use Python version 3.6 or 3.7 to run the inference helper.')

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[[0.7666756  0.23332433]
 [0.90326786 0.09673215]
 [0.89099014 0.10900984]
 [0.85169286 0.14830717]
 [0.9196352  0.08036474]
 [0.9166913  0.08330868]
 [0.88521516 0.11478482]
 [0.90795976 0.09204032]
 [0.8745152  0.12548476]
 [0.88407385 0.11592619]
 [0.9067749  0.09322511]
 [0.8487728  0.15122718]
 [0.8595506  0.14044943]
 [0.8959082  0.10409192]
 [0.91442716 0.08557273]
 [0.5973171  0.40268278]
 [0.91576403 0.084236  ]
 [0.88009965 0.11

## Model Deployment

Being the better performing model, I shall hereby deploy the `VotingEnsemble` model.

In [22]:
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

In [23]:
os.makedirs('./amlmodel', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./amlmodel','automl_best_model_cc.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./amlmodel', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
model=best_run.register_model(
            model_name = 'automl-bestmodel-cc', 
            model_path = './outputs/model.pkl',
            model_framework=Model.Framework.SCIKITLEARN,
            description='Cervical Cancer Prediction'
)

In [25]:
# Download the conda environment file and define the environement
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
myenv = Environment.from_conda_specification(name = 'myenv',
                                             file_path = 'conda_env.yml')

In [26]:
# download the scoring file produced by AutoML
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score_auto_cc.py')

# set inference config
inference_config = InferenceConfig(entry_script= 'score_auto_cc.py',
                                    environment=myenv)

In [27]:
# set Aci Webservice config
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True)

In [28]:
service = Model.deploy(workspace=ws, 
                       name='automl-bestmodel-cc', 
                       models=[model], 
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

In [29]:
service

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-135758', subscription_id='510b94ba-e453-4417-988b-fbdc37b55ca7', resource_group='aml-quickstarts-135758'), name=automl-bestmodel-cc, image_id=None, compute_type=None, state=ACI, scoring_uri=Transitioning, tags=None, properties={}, created_by={})

In [30]:
# wait for deployment to finish and display the scoring uri and swagger uri
service.wait_for_deployment(show_output=True)

print('Service state:')
print(service.state)

print('Scoring URI:')
print(service.scoring_uri)

print('Swagger URI:')
print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..............................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Service state:
Healthy
Scoring URI:
http://731d3ebf-3734-4213-9182-e56b83b97293.southcentralus.azurecontainer.io/score
Swagger URI:
http://731d3ebf-3734-4213-9182-e56b83b97293.southcentralus.azurecontainer.io/swagger.json


In [31]:
import json

# select 3  samples from the dataframe
x_df=df.sample(3)
y_df = x_df.pop('Biopsy')

x_df['Column1'] = 0.0

# convert the records to a json data file
recored=x_df.to_dict(orient='records')

scoring_json = json.dumps({'data': recored})
print(scoring_json)

{"data": [{"Age": 24.0, "Number of sexual partners": 2.0, "First sexual intercourse": 19.0, "Num of pregnancies": 1.0, "Smokes": 0.0, "Smokes (years)": 0.0, "Smokes (packs/year)": 0.0, "Hormonal Contraceptives": 1.0, "Hormonal Contraceptives (years)": 6.0, "IUD": 0.0, "IUD (years)": 0.0, "STDs": 0.0, "STDs (number)": 0.0, "STDs:condylomatosis": 0.0, "STDs:vaginal condylomatosis": 0.0, "STDs:vulvo-perineal condylomatosis": 0.0, "STDs:syphilis": 0.0, "STDs:pelvic inflammatory disease": 0.0, "STDs:genital herpes": 0.0, "STDs:molluscum contagiosum": 0.0, "STDs:HIV": 0.0, "STDs:Hepatitis B": 0.0, "STDs:HPV": 0.0, "STDs: Number of diagnosis": 0.0, "Dx:Cancer": 0.0, "Dx:CIN": 0.0, "Dx:HPV": 0.0, "Dx": 0.0, "Column1": 0.0}, {"Age": 24.0, "Number of sexual partners": 2.0, "First sexual intercourse": 19.0, "Num of pregnancies": 1.0, "Smokes": 0.0, "Smokes (years)": 0.0, "Smokes (packs/year)": 0.0, "Hormonal Contraceptives": 0.0, "Hormonal Contraceptives (years)": 0.0, "IUD": 0.0, "IUD (years)": 

Consumint the endpoint using `endpoint.py`

In [49]:
!python3 endpoint.py

{"result": [0.0, 0.0, 0.0]}


In [34]:
output = service.run(scoring_json)
output

'{"result": [0.0, 0.0, 0.0]}'

In [50]:
y_df

241   0.00
793   0.00
5     0.00
Name: Biopsy, dtype: float64

Enabling logging using `logs.py`.

In [51]:
!python3 logs.py

2021-01-23T10:00:59,679817292+00:00 - iot-server/run 
2021-01-23T10:00:59,682904066+00:00 - gunicorn/run 
2021-01-23T10:00:59,686459651+00:00 - rsyslog/run 
2021-01-23T10:00:59,690376245+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd